Imports

In [ ]:
#imports
from tqdm import tqdm
from transformers import pipeline
from utils import load_data, prepare_all_samples, get_entity_date_pairs, calculate_metrics
from llm_extractor import load_prompt_template, make_binary_prompt, llm_extraction, parse_llm_answer

Data Loading

In [2]:
# Load data
df = load_data("data/synthetic.csv")
print(f"Loaded {len(df)} records")
#df

Loaded 101 records


In [3]:
# Prepare all samples
samples = prepare_all_samples(df)
print(f"Prepared {len(samples)} samples")
#samples[0]

Prepared 101 samples


LLM

In [4]:
# Define generator
#generator = pipeline("text-generation", model="../Llama-3.2-3B-Instruct", device=-1)
generator = pipeline("text2text-generation", model="google/flan-t5-small", device=-1)

Device set to use cpu


In [5]:
# Test simple prompt using generator
prompt = "Does the following text indicate a relationship between 'asthma' and '2024-08-02'? Answer YES or NO. Text: Patient diagnosed with asthma on 2024-08-02."
result = generator(prompt)
print(result[0]['generated_text'])

No


In [6]:
# Test simple prompt using llm_extraction() function
prompt = "Does the following text indicate a relationship between 'asthma' and '2024-08-02'? Answer YES or NO. Text: Patient diagnosed with asthma on 2024-08-02."
response = llm_extraction(prompt, generator)
response

'No'

In [7]:
#Prompt to use
prompt_to_use = 'prompt.txt'

In [ ]:
#Process all date-entity pairs, make prompt, do llm extraction and make prediction
predictions = []

for sample in tqdm(samples, desc="Samples"):
    pairs = get_entity_date_pairs(sample['entities_list'], sample['dates'])
    #for pair in pairs[:1]:
    for pair in pairs:
        #print(pair)
        prompt = make_binary_prompt(pair['entity'], pair['date_info'], sample['note_text'], prompt_to_use)
        #print(prompt)
        response = llm_extraction(prompt, generator)
        #print(response)
        pred, conf = parse_llm_answer(response)
        #print(pred, conf)
        if pred == 1:
            predictions.append({
                'entity_label': pair['entity_label'],
                'date': pair['date'],
                'confidence': conf
            })

Samples:  25%|██▍       | 25/101 [05:14<15:16, 12.06s/it]

In [ ]:
#Look at prediction
predictions

In [ ]:
#Calculcate metrics
metrics = calculate_metrics(predictions, df)
metrics